In [ ]:
from operator import add
import os
from lxml import etree
import numpy as np
from datetime import datetime
from datetime import timedelta

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")

In [ ]:
def isRecord(line):
    if line.startswith("  <row") and line.endswith(" />"):
        return line

In [ ]:
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

In [ ]:
# need a special way to pickle classes
class Record(object):
    def __init__(self, Id, PostId, VoteTypeId):
        self.Id = Id
        self.PostId = PostId
        self.VoteTypeId = VoteTypeId

In [ ]:
def parse(line):
    
    line = line.encode('utf-8')
    parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
    root = etree.fromstring(line, parser=parser)
    
    VoteId = root.attrib['Id']
    PostId = root.attrib['PostId']
    VoteType = root.attrib['VoteTypeId']
    
    # values for [upmod, downmod, favorite]
    if VoteType == '2':
        output = [1, 0, 0]
    elif VoteType == '3':
        output = [0, 1, 0]
    elif VoteType == '5':
        output = [0, 0, 1]
    else:
        output = [0, 0, 0]
    
    return PostId, output

In [ ]:
def addFavs(x, y):
    new_score = map(add, x, y)
    return new_score

In [ ]:
def byFavorate(posts):
    id = posts[0]
    score = posts[1]
    fav = score[2]
    up = float(score[0])
    down = float(score[1])
    if up + down != 0:
        percent = float(up / (up + down))
        return fav, percent
    else:
        return fav, 0

In [ ]:
lines = sc.textFile(localpath('/full/allVotes/part-00000.xml.gz')) \
        .filter(lambda x: isRecord(x)) \
        .map(parse) \
        #.reduceByKey(lambda x, y: map(add, x, y)) \
        #.map(byFavorate) \
        #.aggregateByKey((0,0), lambda a,b: (a[0] + b,    a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1])) \
        #.mapValues(lambda v: v[0]/v[1]) \
        #.sortByKey() \
        
lines.take(10)

In [ ]:
lines = sc.textFile(localpath('/allVotes/*.gz')) \
        .filter(lambda x: isRecord(x)) \
        .map(parse) \
        .reduceByKey(lambda x, y: map(add, x, y)) \
        .map(byFavorate) \
        .aggregateByKey((0,0), lambda a,b: (a[0] + b,    a[1] + 1),
                                      lambda a,b: (a[0] + b[0], a[1] + b[1])) \
        .mapValues(lambda v: v[0]/v[1]) \
        .sortByKey() \
        
lines.take(100)

# Question2

In [ ]:
def parse_rep(line):
    
    # line = line.replace(u"\r", "")
    line = line.encode('utf-8')
    parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
    root = etree.fromstring(line, parser=parser)
    
    rep = root.attrib['Reputation']
    ID = root.attrib['Id']
    
    return ID, int(rep)

In [ ]:
UserID = sc.textFile(localpath('/stats/allUsers/')) \
        .filter(lambda x: isRecord(x)) \
        .map(parse_rep) \
        #.reduceByKey(lambda x, y: map(add, x, y)) \
        #.map(byFavorate) \
        #.aggregateByKey((0,0), lambda a,b: (a[0] + b,    a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1])) \
        #.mapValues(lambda v: v[0]/v[1]) \
        #.sortByKey() \
        
UserID.takeOrdered(99, key = lambda x: -x[1])
# UserID.take(10)

In [ ]:
def parse_postType(line):
    try:
        line = line.replace(u"\r", "")
        line = line.encode('utf-8')
        parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
        root = etree.fromstring(line, parser=parser)

        owner = root.attrib['OwnerUserId']
        post_type = root.attrib['PostTypeId']

        if post_type == '1':
            return owner, [1,0]
        if post_type == '2':
            return owner, [0,1]
    except:
        pass

In [ ]:
def validPost(line):
    if 'OwnerUserId' in line and 'PostTypeId' in line:
        return line

In [ ]:
def calRate(thing):
    answer = float(thing[1][1])
    question = float(thing[1][0])
    rate = float(answer/ (answer + question))
    return thing[0], rate

In [ ]:
post_type = sc.textFile(localpath('/stats/allPosts/*.gz')) \
        .filter(lambda x: validPost(x)) \
        .map(parse_postType) \
        .filter(takeOutNone) \
        .reduceByKey(lambda x, y: map(add, x, y)) \
        .map(calRate)
        
post_type.take(10)

In [ ]:
def takeOutNone(line):
    if line != 'None':
        return line

In [ ]:
UserID = sc.textFile(localpath('/stats/allUsers/*.gz')) \
        .filter(lambda x: isRecord(x)) \
        .map(parse_rep)
post_type = sc.textFile(localpath('/stats/allPosts/*.gz')) \
        .filter(lambda x: isRecord(x)) \
        .map(parse_postType) \
        .filter(takeOutNone) \
        .reduceByKey(lambda x, y: map(add, x, y))

user_answer = UserID.join(post_type)
top_list = user_answer.takeOrdered(99, key = lambda x: -x[1][0])

In [ ]:
top_list

In [ ]:
question2 = []
for i in top_list:
    Id = i[0]
    answer = float(i[1][1][1])
    question = float(i[1][1][0])
    question2.append( (int(Id), float(answer/ (answer + question)) ))

In [ ]:
question2

# Question 3

In [ ]:
def post_count(line):
    try:
        line = line.replace(u"\r", "")
        line = line.encode('utf-8')
        parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
        root = etree.fromstring(line, parser=parser)

        owner = root.attrib['OwnerUserId']
        return owner, 1
    except:
        pass

In [ ]:
def find_average(value):
    count = value[1]
    if count 

In [ ]:
post_type = sc.textFile(localpath('/stats/allPosts/*.gz')) \
        .filter(lambda x: validPost(x)) \
        .map(post_count) \
        .filter(takeOutNone) \
        .reduceByKey(lambda x, y: x+y) \

        
top = post_type.takeOrdered(100, key = lambda x: -x[1])

In [ ]:
top_count = [x[1] for x in top]
# np.mean(top_count)
top

In [ ]:
def take_count(value):
    count = value[1][1]
    rep = float(value[1][0])
    return count, rep

In [ ]:
UserID = sc.textFile(localpath('/stats/allUsers/*.gz')) \
        .filter(lambda x: isRecord(x)) \
        .map(parse_rep)
post_type = sc.textFile(localpath('/stats/allPosts/*.gz')) \
        .filter(lambda x: validPost(x)) \
        .map(post_count) \
        .filter(takeOutNone) \
        .reduceByKey(lambda x, y: x+y) \

user_answer = UserID.join(post_type) \
        .map(take_count) \
        .aggregateByKey((0,0), lambda a,b: (a[0] + b, a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1])) \
        .mapValues(lambda v: v[0]/v[1]) 
top_count = user_answer.takeOrdered(100, key = lambda x: -x[0])

In [ ]:
top_count

# Question 4

In [ ]:
def valid_count(line):
    if 'AcceptedAnswerId' in line and 'CreationDate' in line:
        return line

In [ ]:
def validPost(line):
    if 'OwnerUserId' in line and 'PostTypeId' in line:
        return line

In [ ]:
def parse_answer(line):
    
    try:
        line = line.encode('utf-8')
        parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
        root = etree.fromstring(line, parser=parser)

        # Id = root.attrib['Id']
        answer = root.attrib['AcceptedAnswerId']
        creation_date = root.attrib['CreationDate']

        return int(answer), creation_date
    except:
        pass

In [ ]:
def parse_postID(line):

    try:
        line = line.encode('utf-8')
        parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
        root = etree.fromstring(line, parser=parser)

        Id = root.attrib['Id']
        creation_date = root.attrib['CreationDate']

        return int(Id), creation_date
    except:
        pass

In [ ]:
def get_time(x):
    time1 = x[1][0]
    time2 = x[1][1]
    time1 = datetime.strptime(time1, '%Y-%m-%dT%H:%M:%S.%f')
    time2 = datetime.strptime(time2, '%Y-%m-%dT%H:%M:%S.%f')
    if (time2 - time1).seconds >= 10800:
        # long time
        return time1.hour, [1, 0]
    else:
        # short time
        return time1.hour, [0, 1]

In [ ]:
def calc_ratio(value):
    hour = value[0]
    long_resp = float(value[1][0])
    short_resp = float(value[1][1])
    ratio = short_resp / (short_resp + long_resp)
    return hour, ratio

In [ ]:
def take_second(value):
    second = value[1]
    return second

In [ ]:
answer_time = sc.textFile(localpath('/stats/allPosts/*.gz')) \
        .filter(lambda x: valid_count(x)) \
        .map(parse_answer) \
        .filter(takeOutNone) \

postID_time = sc.textFile(localpath('/stats/allPosts/*.gz')) \
        .filter(lambda x: isRecord(x)) \
        .map(parse_postID) \

match_answer = answer_time.join(postID_time)
match_answer.map(get_time) \
            .reduceByKey(lambda x, y: map(add, x, y)) \
            .map(calc_ratio) \
            .sortByKey() \
            .map(take_second) \
            .take(24)

# match_answer.takeOrdered(10, key = lambda x: x[0])